In [2]:
from xdsl.dialects.experimental import aie
from xdsl.dialects.builtin import Region, IndexType, ModuleOp, i32, IntegerAttr, ArrayAttr, i64, StringAttr
from xdsl.dialects.arith import Constant
from xdsl.dialects.memref import MemRefType
from xdsl.dialects import builtin, arith, memref, func
from xdsl.builder import Builder
from xdsl.traits import SymbolTable

def I32Attr(value):
    return IntegerAttr.from_int_and_width(value, i32)

In [3]:
def tutorial5():
    tile34 = aie.TileOp(I32Attr(3), I32Attr(4))
    tile70 = aie.TileOp(I32Attr(7), I32Attr(0))

    ext_buf70_in = aie.ExternalBufferOp("ddr_test_buffer_in", ArrayAttr([I32Attr(256)]), i32)
    ext_buf70_out = aie.ExternalBufferOp("ddr_test_buffer_out", ArrayAttr([I32Attr(256)]), i32)

    object_fifo_in = aie.createObjectFifo(I32Attr(256), tile70, tile34, i32, [256], "of_in")
    object_fifo_out = aie.createObjectFifo(I32Attr(256), tile34, tile70, i32, [256], "of_out")

    register_of_in = aie.ObjectFifoRegisterExternalBuffersOp(tile70, ext_buf70_in, object_fifo_in.sym_name)
    register_of_out = aie.ObjectFifoRegisterExternalBuffersOp(tile70, ext_buf70_out, object_fifo_out.sym_name)


    @Builder.region
    def region0(builder: Builder):
        builder.insert(tile34)
        builder.insert(tile70)
        builder.insert(ext_buf70_in)
        builder.insert(ext_buf70_out)
        builder.insert(object_fifo_in)
        builder.insert(object_fifo_out)
        builder.insert(register_of_in)
        builder.insert(register_of_out)

    device = aie.DeviceOp(I32Attr(0), region0)

    # CORE 34
    acquire_fifo_of_in = aie.ObjectFifoAcquireOp(I32Attr(aie.CONSUME_PORT), I32Attr(1), object_fifo_in.sym_name, device)
    acquire_fifo_of_out = aie.ObjectFifoAcquireOp(I32Attr(aie.PRODUCE_PORT), I32Attr(1), object_fifo_out.sym_name, device)

    subview_of_in = aie.ObjectFIFOSubviewAccessOp(I32Attr(0), acquire_fifo_of_in)
    subview_of_out = aie.ObjectFIFOSubviewAccessOp(I32Attr(0), acquire_fifo_of_out)

    idx1 = arith.Constant.from_int_and_width(3, IndexType())
    d1 = memref.Load.get(subview_of_in, idx1)
    c1 = arith.Constant.from_int_and_width(100, i32)
    d2 = arith.Addi(d1, c1)
    idx2 = arith.Constant.from_int_and_width(5, IndexType())
    memref_store_d2_in = memref.Store.get(d2, subview_of_in, idx2)
    memref_store_d1_out = memref.Store.get(d1, subview_of_out, idx1)
    memref_store_d2_out = memref.Store.get(d2, subview_of_out, idx2)

    release_fifo_in = aie.ObjectFIFOReleaseOp(I32Attr(aie.CONSUME_PORT), I32Attr(1), object_fifo_in.sym_name)
    release_fifo_out = aie.ObjectFIFOReleaseOp(I32Attr(aie.PRODUCE_PORT), I32Attr(1), object_fifo_out.sym_name)

    @Builder.region
    def core34_region(builder: Builder):
        builder.insert(acquire_fifo_of_in)
        builder.insert(acquire_fifo_of_out)
        builder.insert(subview_of_in)
        builder.insert(subview_of_out)
        builder.insert(idx1)
        builder.insert(d1)
        builder.insert(c1)
        builder.insert(d2)
        builder.insert(idx2)
        builder.insert(memref_store_d2_in)
        builder.insert(memref_store_d1_out)
        builder.insert(memref_store_d2_out)
        builder.insert(release_fifo_in)
        builder.insert(release_fifo_out)
        builder.insert(aie.EndOp())

    core34 = aie.CoreOp(I32Attr(1), tile34, core34_region)
    
    region0.block.add_op(core34)

    module = ModuleOp([device])

    print(module)

In [4]:
tutorial5()

builtin.module {
  AIE.device(xcvc1902) {
    %0 = "AIE.tile"() {"col" = 3 : i32, "row" = 4 : i32} : () -> index
    %1 = "AIE.tile"() {"col" = 7 : i32, "row" = 0 : i32} : () -> index
    %2 = "AIE.external_buffer"() {"sym_name" = "ddr_test_buffer_in"} : () -> memref<256xi32>
    %3 = "AIE.external_buffer"() {"sym_name" = "ddr_test_buffer_out"} : () -> memref<256xi32>
    AIE.objectFifo @of_in(%1, {%0}, 256 : i32) : !AIE.objectFifo<memref<256xi32>>
    AIE.objectFifo @of_out(%0, {%1}, 256 : i32) : !AIE.objectFifo<memref<256xi32>>
    AIE.objectFifo.registerExternalBuffers @of_in (%1, {%2}) : (memref<256xi32>)
    AIE.objectFifo.registerExternalBuffers @of_out (%1, {%3}) : (memref<256xi32>)
    %4 = AIE.core (%0) {
      %5 = AIE.objectFifo.acquire @of_in (Consume, 1) : !AIE.objectFifoSubview<memref<256xi32>>
      %6 = AIE.objectFifo.acquire @of_out (Produce, 1) : !AIE.objectFifoSubview<memref<256xi32>>
      %7 = AIE.objectFifo.subview.access %5[0] : !AIE.objectFifoSubview<memref<256x